<a href="https://colab.research.google.com/github/Parikshit07111996/School-Of-AI/blob/master/Assignment_File_Session_5_(2)_Basic_Skeleton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Target
1. Get the basic skeleton of the model 
2. Decrease number of parameters

# Results
1. Parameters : 195k
2. Best Training Accuracy : 99.52
3. Best Test Accuracy : 99.62

#Analysis
1. Extremely Heavy Model for Mnist data
2. Model is clearly over-fitting 

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

#Data Transform



In [0]:
# Train Phase Transformation
train_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,),(0.3081,))
                                       ])


# Test Phase Transformation
test_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,),(0.3081,))
                                       ])

# Dataset and Creating Train/Test Split

In [3]:
train = datasets.MNIST('./data',train=True, download=True, transform= train_transforms)
test = datasets.MNIST('./data',train=True, download=True, transform= test_transforms)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


#Dataloader Arguments & Test/Train Dataloaders

In [0]:
SEED = 1

cuda = torch.cuda.is_available()

torch.manual_seed(SEED)

if cuda:
  torch.cuda.manual_seed(SEED)

dataloader_args = dict(shuffle=True, batch_size= 128, num_workers=4, pin_memory=True)

train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

test_loader = torch.utils.data.DataLoader(test, **dataloader_args)


#The Model

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #Input Block
        self.convblock1 = nn.Sequential(
          nn.Conv2d(1, 32, 3, 1,False),
          nn.ReLU()     
         ) # Output size = 26
        
        #Convolution Block 1
        self.convblock2 = nn.Sequential(
        nn.Conv2d(32, 64, 3,1,False),
        nn.ReLU()     
        ) # Output size = 24

        self.convblock3 = nn.Sequential(
        nn.Conv2d(64, 128, 3,1,False),
        nn.ReLU()     
        ) # Output size = 22

        #Transition Block 1

        self.pool1 = nn.MaxPool2d(2, 2) # Outout_size = 11

        self.convblock4 = nn.Sequential(
            nn.Conv2d(128, 32, 1),
            nn.ReLU()
        ) # Outout_size = 11

        # Convolution Block 2       
        self.convblock5 = nn.Sequential(
        nn.Conv2d(32, 64, 3),
        nn.ReLU()    
        ) # output size 9

        self.convblock6 = nn.Sequential(
        nn.Conv2d(64, 128, 3),
        nn.ReLU()    
        ) # output size 7

        # Output Block 
        self.convblock7 = nn.Sequential(
        nn.Conv2d(128, 10, 1),
        nn.ReLU()
        )  # output size 7

        self.convblock8 = nn.Sequential(
        nn.Conv2d(10, 10, 7),
        )  # output size 1
        
        
        

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.pool1(x)
        x = self.convblock4(x)
        x = self.convblock5(x)
        x = self.convblock6(x)
        x = self.convblock7(x)
        x = self.convblock8(x)
        
        x = x.view(-1, 10)
        return F.log_softmax(x)

#Model Parameters

In [23]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
              ReLU-2           [-1, 32, 26, 26]               0
            Conv2d-3           [-1, 64, 24, 24]          18,496
              ReLU-4           [-1, 64, 24, 24]               0
            Conv2d-5          [-1, 128, 22, 22]          73,856
              ReLU-6          [-1, 128, 22, 22]               0
         MaxPool2d-7          [-1, 128, 11, 11]               0
            Conv2d-8           [-1, 32, 11, 11]           4,128
              ReLU-9           [-1, 32, 11, 11]               0
           Conv2d-10             [-1, 64, 9, 9]          18,496
             ReLU-11             [-1, 64, 9, 9]               0
           Conv2d-12            [-1, 128, 7, 7]          73,856
             ReLU-13            [-1, 128, 7, 7]               0
           Conv2d-14             [

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:66: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


#Training and Testing

In [0]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []



def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
      #get samples
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        y_pred = model(data)
        loss = F.nll_loss(y_pred, target)
        loss.backward()
        optimizer.step()

        pred = y_pred.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)
        pbar.set_description(desc= f'loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
        train_acc.append(100*correct/processed)


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

#Train & Test Our Model

In [25]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 20

for epoch in range(EPOCHS):
  print("EPOCH:", epoch)
  train(model, device, train_loader, optimizer, epoch)
  test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

EPOCH: 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:66: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.0973917618393898 Batch_id=468 Accuracy=62.21: 100%|██████████| 469/469 [00:11<00:00, 40.37it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1837, Accuracy: 56672/60000 (94.45%)

EPOCH: 1


loss=0.09070812910795212 Batch_id=468 Accuracy=95.89: 100%|██████████| 469/469 [00:11<00:00, 40.35it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0157, Accuracy: 59710/60000 (99.52%)

EPOCH: 18


loss=0.00454142689704895 Batch_id=468 Accuracy=99.51: 100%|██████████| 469/469 [00:11<00:00, 39.83it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0166, Accuracy: 59658/60000 (99.43%)

EPOCH: 19


loss=0.0017702182522043586 Batch_id=468 Accuracy=99.52: 100%|██████████| 469/469 [00:11<00:00, 40.29it/s]



Test set: Average loss: 0.0119, Accuracy: 59773/60000 (99.62%)

